# **Main**

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [2]:
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.3 MB/s eta 0:00:00


In [3]:
!pip install opencv-python numpy torch

In [4]:
%cd '/content/drive/MyDrive/2025/KratiTech/UPSIDA/Smart Surveillance /Netra/Ghaziabad/Cam 1'

/content/drive/MyDrive/2025/KratiTech/UPSIDA/Smart Surveillance /Netra/Ghaziabad/Cam 1


In [5]:
from ultralytics  import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import numpy as np
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
video_path = 'samples/9th June/5am/combined_video.mp4'
output_video_path = "samples/9th June/5am/tracked_video.mp4"

if not os.path.exists(video_path):
    print(f"Error: Video file not found at {video_path}")
else:
    print(f"Loading video from: {video_path}")

Loading video from: samples/9th June/5am/combined_video.mp4


In [7]:
print("Loading AI model...")
model = YOLO('yolov8n.pt')

Loading AI model...


In [8]:
cctv_feed = cv2.VideoCapture(video_path)

if not cctv_feed.isOpened():
  print("Error: Could not open video.")
else:
  print("Video loaded successfully. Processing frames...")

Video loaded successfully. Processing frames...


In [9]:
tracker = DeepSort(max_age=30, n_init=5)

In [10]:
frame_width = int(cctv_feed.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cctv_feed.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cctv_feed.get(cv2.CAP_PROP_FPS))

print(fps)

frame_count = 0
frames_to_skip = int(fps)
frame_count = 0
t=1

tracked_summary = {}

14


In [11]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

if not out_video.isOpened():
  print(f"Error: Could not initialize VideoWriter for output file: {output_video_path}")
  cctv_feed.release()

print(f"Processing video... Output will be saved to {output_video_path}")

Processing video... Output will be saved to samples/9th June/5am/tracked_video.mp4


In [12]:
while cctv_feed.isOpened():

    ret, frame = cctv_feed.read()

    if not ret:
        break  # Stop when video ends

    print("Logic frame count :",frame_count)
    print("Analyzing image for objects...")
    flag = False

    # Run YOLO detection on the current frame
    results = model(frame)

    # Store detected persons
    detections = []
    class_names = results[0].names
    i=1
    print("Processing detections...")

    for r in results[0].boxes.data:
      x1, y1, x2, y2, conf, cls = r

      detections.append(([x1, y1, x2, y2], conf,int(cls)))

    print(detections)

    # Pass detections to DeepSORT tracker
    tracked_objects = tracker.update_tracks(detections, frame=frame)

    for obj in tracked_objects:
      if obj.is_confirmed():
          s="object"+str(i)
          i+=1

          if t not in tracked_summary:
              tracked_summary[t] = {}

          tracked_summary[t][s] = {"class":class_names[obj.det_class],"track_id":obj.track_id,"confidence_score":obj.det_conf,"xyxy":obj.to_tlbr()}
          print(f"Class {class_names[obj.det_class]} : ID {obj.track_id}: Confidence {obj.det_conf} : Bounding Box {obj.to_tlbr()}")
          bbox = obj.to_tlbr()  # Convert bbox to (top-left, bottom-right)
          id = obj.track_id
          cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
          cv2.putText(frame, f"ID {id}", (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    out_video.write(frame)
    t+=1
    frame_count += 1

Logic frame count : 0
Analyzing image for objects...

0: 384x640 2 persons, 1 train, 399.1ms
Speed: 8.0ms preprocess, 399.1ms inference, 31.9ms postprocess per image at shape (1, 3, 384, 640)
Processing detections...
[([tensor(1722.6960), tensor(946.8572), tensor(1862.5911), tensor(1307.5427)], tensor(0.7159), 0), ([tensor(1852.9315), tensor(974.7105), tensor(1921.6256), tensor(1075.0234)], tensor(0.4063), 0), ([tensor(0.0728), tensor(558.1110), tensor(752.7139), tensor(934.6522)], tensor(0.3463), 6)]
Logic frame count : 1
Analyzing image for objects...

0: 384x640 3 persons, 1 bicycle, 1 train, 1 boat, 285.9ms
Speed: 4.7ms preprocess, 285.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing detections...
[([tensor(1727.4556), tensor(941.9409), tensor(1858.5703), tensor(1309.0864)], tensor(0.6741), 0), ([tensor(0.6456), tensor(559.2654), tensor(749.8453), tensor(939.2368)], tensor(0.4019), 6), ([tensor(1729.9719), tensor(1154.4902), tensor(1904.4612), tensor(

In [13]:
print(tracked_summary)

{5: {'object1': {'class': 'person', 'track_id': '1', 'confidence_score': 0.7236363291740417, 'xyxy': array([     1722.3,      938.46,      3585.7,      2247.2])}, 'object2': {'class': 'person', 'track_id': '2', 'confidence_score': 0.2673659324645996, 'xyxy': array([     1859.1,       972.4,      3775.7,        2045])}, 'object3': {'class': 'train', 'track_id': '3', 'confidence_score': 0.3377344608306885, 'xyxy': array([     3.6177,      559.11,      761.45,      1500.2])}}, 6: {'object1': {'class': 'person', 'track_id': '1', 'confidence_score': 0.6964578628540039, 'xyxy': array([     1723.5,      937.35,        3587,      2246.2])}, 'object2': {'class': 'bicycle', 'track_id': '2', 'confidence_score': 0.28645315766334534, 'xyxy': array([     1584.1,        1048,      3946.3,      2384.6])}, 'object3': {'class': 'train', 'track_id': '3', 'confidence_score': 0.4098939597606659, 'xyxy': array([    -1.3403,      557.18,      755.49,      1497.6])}}, 7: {'object1': {'class': 'person', 'track

In [14]:
# Cleanup
cctv_feed.release()
out_video.release()

In [15]:
import csv
import datetime

In [16]:
csv_filepath = 'samples/9th June/5am/tracked_objects.csv'
write_header = not os.path.exists(csv_filepath)

In [17]:
with open(csv_filepath, mode='w', newline='') as csv_file:
  writer = csv.writer(csv_file)

  # Write header if file is new
  if write_header:
    writer.writerow(['Frames', 'Class', 'Track Id','Confidence Score','XYXY'])

  # Write data from tracked_summary
  for frame, objects in tracked_summary.items():
    for obj_name, obj_data in objects.items():
      writer.writerow([
          frame,
          obj_data['class'],
          obj_data['track_id'],
          obj_data['confidence_score'],
          str(obj_data['xyxy'])
      ])

print(f"Tracked summary written to {csv_filepath}")

Tracked summary written to samples/9th June/5am/tracked_objects.csv
